# Install Dependencies

In [1]:
! conda install numpy pandas matplotlib -y

! pip install arrow
! pip install requests
! pip install shapely

# For connecting to the DB (used by our scripts)
! conda install -c anaconda psycopg2 -y
! pip install sqlalchemy
! pip install geoalchemy2

Fetching package metadata ...........
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /home/benjiao/anaconda3/envs/gme205-notebooks:
#
matplotlib                2.1.0            py36hba5de38_0  
numpy                     1.13.3           py36ha12f23b_0  
pandas                    0.21.0           py36h78bd809_1  
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /home/benjiao/anaconda3/envs/gme205-notebooks:
#
psycopg2                  2.7.3.1          py36h369a60c_0    anaconda


# Fetch data from NGDC

> National Geophysical Data Center / World Data Service (NGDC/WDS): Significant Earthquake Database. National Geophysical Data Center, NOAA. doi:10.7289/V5TD9V7K

In [2]:
import requests
results = requests.get("https://www.ngdc.noaa.gov/nndc/struts/results?type_0=Exact&query_0=$ID&t=101650&s=13&d=189&dfn=signif.txt")

# Read CSV as a Python Dictionary

In [3]:
import csv
from io import StringIO

f = StringIO(results.text)
reader = csv.DictReader(f, delimiter='\t')
earthquakes = [row for row in reader]

# Load Data into Pandas for easier inspection

In [4]:
import pandas
earthquakes_df = pandas.DataFrame.from_dict(earthquakes)
earthquakes_df.tail()

,I_D,FLAG_TSUNAMI,YEAR,MONTH,DAY,HOUR,MINUTE,SECOND,FOCAL_DEPTH,EQ_PRIMARY,...,TOTAL_MISSING,TOTAL_MISSING_DESCRIPTION,TOTAL_INJURIES,TOTAL_INJURIES_DESCRIPTION,TOTAL_DAMAGE_MILLIONS_DOLLARS,TOTAL_DAMAGE_DESCRIPTION,TOTAL_HOUSES_DESTROYED,TOTAL_HOUSES_DESTROYED_DESCRIPTION,TOTAL_HOUSES_DAMAGED,TOTAL_HOUSES_DAMAGED_DESCRIPTION
5971,10275,,2017,11,12,18,18,17.0,23,7.3,...,,,9818,4,,,15500,4,14500,4
5972,10276,,2017,11,13,2,28,24.0,20,6.5,...,,,,,,,,,,
5973,10277,,2017,11,15,5,29,32.0,10,5.4,...,,,57,2,,2,,,1001,4
5974,10282,,2017,11,18,16,7,2.0,20,5.9,...,,,,,,2,,,294,3
5975,10281,Tsu,2017,11,19,22,43,29.0,10,7.0,...,,,,,,,,,,


List column names

In [5]:
list(earthquakes_df)

['I_D',
 'FLAG_TSUNAMI',
 'YEAR',
 'MONTH',
 'DAY',
 'HOUR',
 'MINUTE',
 'SECOND',
 'FOCAL_DEPTH',
 'EQ_PRIMARY',
 'EQ_MAG_MW',
 'EQ_MAG_MS',
 'EQ_MAG_MB',
 'EQ_MAG_ML',
 'EQ_MAG_MFA',
 'EQ_MAG_UNK',
 'INTENSITY',
 'COUNTRY',
 'STATE',
 'LOCATION_NAME',
 'LATITUDE',
 'LONGITUDE',
 'REGION_CODE',
 'DEATHS',
 'DEATHS_DESCRIPTION',
 'MISSING',
 'MISSING_DESCRIPTION',
 'INJURIES',
 'INJURIES_DESCRIPTION',
 'DAMAGE_MILLIONS_DOLLARS',
 'DAMAGE_DESCRIPTION',
 'HOUSES_DESTROYED',
 'HOUSES_DESTROYED_DESCRIPTION',
 'HOUSES_DAMAGED',
 'HOUSES_DAMAGED_DESCRIPTION',
 'TOTAL_DEATHS',
 'TOTAL_DEATHS_DESCRIPTION',
 'TOTAL_MISSING',
 'TOTAL_MISSING_DESCRIPTION',
 'TOTAL_INJURIES',
 'TOTAL_INJURIES_DESCRIPTION',
 'TOTAL_DAMAGE_MILLIONS_DOLLARS',
 'TOTAL_DAMAGE_DESCRIPTION',
 'TOTAL_HOUSES_DESTROYED',
 'TOTAL_HOUSES_DESTROYED_DESCRIPTION',
 'TOTAL_HOUSES_DAMAGED',
 'TOTAL_HOUSES_DAMAGED_DESCRIPTION']

## Format rows

Let's prepare our rows for saving into Postgres.

In [6]:
from shapely.geometry import Point
from shapely.wkt import dumps
import datetime


def int_or_none(val):
    return int(val) if val is not "" else None

def float_or_none(val):
    return float(val) if val is not "" else None

def format_row(row):
    
    # Format location field into WKT
    try:
        location = dumps(Point(float(row['LONGITUDE']), float(row['LATITUDE'])))
    except (AttributeError, ValueError):
        location = None
    
    # Create date object, none if missing  fields

    try:
        earthquake_date = datetime.date(
            int_or_none(row["YEAR"]),
            int_or_none(row["MONTH"]),
            int_or_none(row["DAY"])
        ).isoformat()
    except:
        earthquake_date = None

    return {
        'id': row["I_D"],
        'name': row["LOCATION_NAME"],
        'date': earthquake_date,
        'location': location,
        'hour': int_or_none(row["HOUR"]),
        'year': int_or_none(row["YEAR"]),
        'month': int_or_none(row["MONTH"]),
        'day': int_or_none(row["DAY"]),
        'minute': int_or_none(row["MINUTE"]),
        'second': float_or_none(row["DAY"]),
        'focal_depth': float_or_none(row['FOCAL_DEPTH']),
        'magnitude': float_or_none(row['EQ_MAG_MW']), 
        'intensity': float_or_none(row['INTENSITY']),
        'deaths': int_or_none(row["DEATHS"]),
        'missing': int_or_none(row["MISSING"]),
        'damage': float_or_none(row["DAMAGE_MILLIONS_DOLLARS"]),
        'houses_destroyed': int_or_none(row['HOUSES_DESTROYED']),
        'houses_damaged': int_or_none(row['HOUSES_DAMAGED']),
        'total_deaths': int_or_none(row["TOTAL_DEATHS"]),
        'total_missing': int_or_none(row["TOTAL_MISSING"]),
        'total_damage': float_or_none(row["TOTAL_DAMAGE_MILLIONS_DOLLARS"]),
        'total_houses_destroyed': int_or_none(row['TOTAL_HOUSES_DESTROYED']),
        'total_houses_damaged': int_or_none(row['TOTAL_HOUSES_DAMAGED'])
    }

earthquakes2 = [format_row(row) for row in earthquakes]
earthquakes_df2 = pandas.DataFrame.from_dict(earthquakes2)
earthquakes_df2.tail(5)  

,damage,date,day,deaths,focal_depth,hour,houses_damaged,houses_destroyed,id,intensity,...,missing,month,name,second,total_damage,total_deaths,total_houses_damaged,total_houses_destroyed,total_missing,year
5971,NaN,2017-11-12,12.0,540.0,23.0,18.0,14500.0,15500.0,10275,NaN,...,NaN,11.0,IRAN: KERMANSHAH; IRAQ: KURDISTAN,12.0,NaN,540.0,14500.0,15500.0,NaN,2017
5972,NaN,2017-11-13,13.0,2.0,20.0,2.0,NaN,NaN,10276,NaN,...,NaN,11.0,COSTA RICA: JACO,13.0,NaN,2.0,NaN,NaN,NaN,2017
5973,NaN,2017-11-15,15.0,NaN,10.0,5.0,1001.0,NaN,10277,NaN,...,NaN,11.0,SOUTH KOREA: POHANG,15.0,NaN,NaN,1001.0,NaN,NaN,2017
5974,NaN,2017-11-18,18.0,1.0,20.0,16.0,294.0,NaN,10282,NaN,...,NaN,11.0,INDONESIA: NORTH MALUKU: MOROTAI,18.0,NaN,1.0,294.0,NaN,NaN,2017
5975,NaN,2017-11-19,19.0,NaN,10.0,22.0,NaN,NaN,10281,NaN,...,NaN,11.0,NEW CALEDONIA: LOYALTY ISLANDS,19.0,NaN,NaN,NaN,NaN,NaN,2017


# Save into CSV File

In [7]:
earthquakes_df2.to_csv("earthquakes/data/earthquakes.csv", index=False)